# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
# add csv to save documents into csv
# add os to write to csv
import csv
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
# base URL from openweathermap.com
url = "http://api.openweathermap.org/data/2.5/weather?"

# query url without city
query_url = url + "appid=" + api_key + "&units=imperial" + "&q="
query_url

In [ ]:
# test query url on a random city (Tokyo)
city = "tokyo"
test_query_url = query_url + city

# convert to .json() to see all the variables
# from this list, we need:
# temp
# humidity
# clouds
# wind
weather_response = requests.get(test_query_url)
weather_json = weather_response.json()
weather_json

In [ ]:
# build lists to hold .append data
city_list = []
cloudiness_list = []
country_list = []
date_list = []
humidity_list = []
lat_list = []
lng_list = []
max_temp_list = []
wind_speed_list = []

# set counters for records (50 records), and then for sets (varies)
record_count = 1
set_count = 1

# Specify the file to txt to
output_path = os.path.join("..", "Images", "What_Is_The_Weather_Like.txt")
# Open the file using "write" mode. Specify the variable to hold the contents
with open(output_path, 'w', newline ='') as txtfile:
    # Initialize txt.writer
    #txtwriter = txt.writer(txtfile)
    
    txtfile.write("Beginning Data Retrieval \n")
    txtfile.write("----------------------------- \n")

    print("Beginning Data Retrieval")
    print("-----------------------------")
                       
    # for loop
    # "i" is the variable that is going to hold the numerical value of what city you're in (integer value)
    # cities is going to represent the value (string value)
    for i, city in enumerate(cities):
        # using modulus to record set_count and record_count
        # if i goes into 50 evenly, and i is >= 50 (use this to create sets and to restart record_count back to 0
        if(i % 50 == 0 and i >= 50):
            set_count += 1
            record_count = 0
    
        # set record_count
        record_count += 1
    
        # build query
        city_url = query_url + city
        print(f'Processing Record {record_count} of Set {set_count} | {city}')
    
        txtfile.write(f'Processing Record {record_count} of Set {set_count} | {city} \n')
        # start try & except statement
        try:
            response = requests.get(query_url + city).json()
            city_list.append(response["name"])
            cloudiness_list.append(response["clouds"]["all"])
            country_list.append(response["sys"]["country"])
            date_list.append(response["dt"])
            humidity_list.append(response["main"]["humidity"])
            lat_list.append(response["coord"]["lat"])
            lng_list.append(response["coord"]["lon"])
            max_temp_list.append(response["main"]["temp_max"])
            wind_speed_list.append(response["wind"]["speed"])
    
        # except statement
        # need the pass to continue on
        except:
            print("City not found. Skipping...")
            txtfile.write("City not found. Skipping...")
            pass

    print("-----------------------------")
    print("Data Retrieval Complete")
    print("-----------------------------")    

    txtfile.write("----------------------------- \n")
    txtfile.write("Data Retrieval Complete \n")
    txtfile.write("-----------------------------")
    
# create a dictionary of all the list
# should be 9 columns total
weather_dictionary = {"City": city_list,
                      "Cloudiness": cloudiness_list,
                      "Country": country_list,
                      "Date": date_list,
                      "Humidity": humidity_list,
                      "Lat": lat_list,
                      "Lng": lng_list,
                      "Max_Temp": max_temp_list,
                      "Wind_Speed": wind_speed_list}

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# convert dictionary into a data frame
weather_df = pd.DataFrame(weather_dictionary)

In [ ]:
# verify the number of rows in the weather_df
# this number should be less than the number in cities due to missing cities
weather_df.count()

In [ ]:
# display the weater_df to look at all the columns
weather_df.head()

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
# set latitude and max temperature
lat_coordinates = weather_df["Lat"]
max_temperature = weather_df["Max_Temp"]

In [ ]:
# create a scatter plot based upon the above data
plt.scatter(lat_coordinates, max_temperature, marker="o", facecolors="steelblue", edgecolors="black")

# Create a title, x label, and y label for our chart
plt.title("City Latitude vs. Max Temperature (07/20/19)")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")
plt.grid()

# Save an image of the chart and print to screen
plt.savefig("../Images/Latitude_vs_Temp_Plot.png")
plt.show()

#### Latitude vs. Humidity Plot

In [ ]:
# set latitude and Humidity
lat_coordinates = weather_df["Lat"]
humidity_list = weather_df["Humidity"]

In [ ]:
# create a scatter plot based upon the above data
plt.scatter(lat_coordinates, humidity_list, marker="o", facecolors="cyan", edgecolors="black")

# Create a title, x label, and y label for our chart
plt.title("City Latitude vs. Humidity (07/20/19)")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.grid()

# Save an image of the chart and print to screen
plt.savefig("../Images/Latitude_vs_Humidity_Plot.png")
plt.show()

#### Latitude vs. Cloudiness Plot

In [ ]:
# set latitude and Cloudiness
lat_coordinates = weather_df["Lat"]
cloudiness_list = weather_df["Cloudiness"]

In [ ]:
# create a scatter plot based upon the above data
plt.scatter(lat_coordinates, cloudiness_list, marker="o", facecolors="darkseagreen", edgecolors="black")

# Create a title, x label, and y label for our chart
plt.title("City Latitude vs. Cloudiness (07/20/19)")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.grid()

# Save an image of the chart and print to screen
plt.savefig("../Images/Latitude_vs_Cloudiness_Plot.png")
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
# set latitude and Wind Speed
lat_coordinates = weather_df["Lat"]
wind_speed_list = weather_df["Wind_Speed"]

In [ ]:
# create a scatter plot based upon the above data
plt.scatter(lat_coordinates, wind_speed_list, marker="o", facecolors="magenta", edgecolors="black")

# Create a title, x label, and y label for our chart
plt.title("City Latitude vs. Wind Speed (07/20/19)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.grid()

# Save an image of the chart and print to screen
plt.savefig("../Images/Latitude_vs_Wind_Speed_Plot.png")
plt.show()